#### SYS ID: Finding the params

In [2]:
from moldy.case_studies.grub_sim.model.grub_sim_sys_id_wrapper import GrubSimSysIdWrapper
from lmfit import Model, Parameter
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

grub_model = GrubSimSysIdWrapper()

/home/student/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-03-21 12:52:31,038	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-03-21 12:52:31,117	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [4]:
#%% import the data
# file_path = "/home/student/catkin_ws/src/data/10_min_data/nempc_grub_hw_10_min_Dataset/nempc_grub_hw_10_min_Dataset.csv"
file_path = "/home/student/catkin_ws/src/data/5_min_data/nempc_grub_hw_5_min_Dataset/nempc_grub_hw_5_min_Dataset.csv"

data_df = pd.read_csv(file_path)
time = data_df["time"].to_numpy().reshape((-1,1))
u = data_df["u"].to_numpy().reshape((-1,1))
v = data_df["v"].to_numpy().reshape((-1,1))
udot = data_df["u_dot"].to_numpy().reshape((-1,1))
vdot = data_df["v_dot"].to_numpy().reshape((-1,1))
uddot = data_df["u_ddot"].to_numpy().reshape((-1,1))
vddot = data_df["v_ddot"].to_numpy().reshape((-1,1))
p0 = data_df["p0"].to_numpy().reshape((-1,1))
p1 = data_df["p1"].to_numpy().reshape((-1,1))
p2 = data_df["p2"].to_numpy().reshape((-1,1))
p3 = data_df["p3"].to_numpy().reshape((-1,1))
p0dot = data_df["p0_dot"].to_numpy().reshape((-1,1))
p1dot = data_df["p1_dot"].to_numpy().reshape((-1,1))
p2dot = data_df["p2_dot"].to_numpy().reshape((-1,1))
p3dot = data_df["p3_dot"].to_numpy().reshape((-1,1))
pc0 = data_df["pc0"].to_numpy().reshape((-1,1))
pc1 = data_df["pc1"].to_numpy().reshape((-1,1))
pc2 = data_df["pc2"].to_numpy().reshape((-1,1))
pc3 = data_df["pc3"].to_numpy().reshape((-1,1))

In [5]:
#%% Define state and input vectors 
x_truth = np.hstack((p0, p1, p2, p3, udot, vdot, u, v))
xdot_truth = np.hstack((p0dot, p1dot, p2dot, p3dot, uddot, vddot, udot, vdot)) 
# xdot_truth = grub_model.scale_xdot(xdot_truth) #scaled -1 to 1

u = np.hstack((pc0, pc1, pc2, pc3))

print(f'Size of x: {np.shape(x_truth)}')
print(f'Size of xdot: {np.shape(xdot_truth)}')
print(f'Size of u: {np.shape(u)}')

Size of x: (30144, 8)
Size of xdot: (30144, 8)
Size of u: (30144, 4)


In [6]:
# create the fitting model
model = Model(grub_model.curvefit_wrapper, independent_vars=['x', 'u'])

print(f'Parameters to Optimize: {model.param_names}')
print(f'Independent Variables: {model.independent_vars}')

print(f'\n\nMax vals: {grub_model.max_vals}')
print(f'Min vals: {grub_model.min_vals}')

Parameters to Optimize: ['stiffness', 'damping', 'alpha']
Independent Variables: ['x', 'u']


Max vals: {'stiffness': 50, 'damping': 50, 'alpha': 50}
Min vals: {'stiffness': 0, 'damping': 0, 'alpha': 1}


In [7]:
#%% Run the fit on the model
numStarts = 1
nonlinear_fit = [None]*numStarts

for i in range(numStarts):
    start_val = np.random.uniform(-1, 1, 6)
    nonlinear_fit[i] = model.fit(
        xdot_truth, x=x_truth, u=u,
        # normalized parameters to find. 
        stiffness=Parameter('stiffness', value=start_val[0], min=-1, max=1),
        damping=Parameter('damping', value=start_val[1], min=-1, max=1),
        alpha=Parameter('alpha', value=start_val[2], min=-1, max=1),
        method="least_squares",
    )    
    print(f"Run {i} R^2 value: {nonlinear_fit[i].rsquared}")
     

Run 0 R^2 value: -2.5363514030671443


In [8]:
#%% Unscale the params
for i in range(numStarts):
    scaled_params = {'stiffness': nonlinear_fit[i].params['stiffness'].value, 'damping':nonlinear_fit[i].params['damping'].value, 'alpha':nonlinear_fit[i].params['alpha'].value}
    print(f'scaled params for run = {i}: {scaled_params}')
for i in range(numStarts):
    unscaled_params = grub_model.unscale_params(scaled_params)
    print(f'Unscaled Params for run = {i}: {unscaled_params}')

scaled params for run = 0: {'stiffness': -0.4372490845664074, 'damping': -0.8906989827477998, 'alpha': -0.9242563743610844}
Unscaled Params for run = 0: {'stiffness': 14.068772885839815, 'damping': 2.732525431305005, 'alpha': 2.8557188281534316}


#### SYS ID: Validating control performance

In [9]:
# functions for saving sim data to a csv
def save_to_csv(x_history, u_history, horizon, sim_time, file_path):
    """
    This function saves the simulated data to a csv file
    """
    time = np.linspace(0, sim_time, horizon) #.reshape((-1,1))
    p0 = x_history[:,0]
    p1 = x_history[:,1]
    p2 = x_history[:,2]
    p3 = x_history[:,3]
    u_dot = x_history[:,4]
    v_dot = x_history[:,5]
    u = x_history[:,6]
    v = x_history[:,7]

    pc0 = u_history[:,0]
    pc1 = u_history[:,1]
    pc2 = u_history[:,2]
    pc3 = u_history[:,3]

    # dirty derivative ok? Since there isn't any noise in this data
    p0_dot, p1_dot, p2_dot, p3_dot = dirty_derivative_pressures(time, p0, p1, p2, p3)
    u_ddot, v_ddot = dirty_derivative_bend_angles(time, u_dot, v_dot)

    df = pd.DataFrame({
    'time': time, 'p0': p0,'p1': p1,'p2': p2,'p3': p3,'u_dot': u_dot,'v_dot': v_dot,
    'u': u,'v': v,'pc0': pc0,'pc1': pc1,'pc2': pc2,'pc3': pc3,'p0_dot': p0_dot,'p1_dot': p1_dot,'p2_dot': p2_dot,'p3_dot': p3_dot,
    'u_ddot': u_ddot,'v_ddot': v_ddot
    })
    
    print(f'shape of df: {np.shape(df)}')
    print(f'df columns: {df.columns}')

    df.to_csv(file_path, index=False)


def dirty_derivative_pressures(pressure_t, p0, p1, p2, p3):
    """
    This function calculates the dirty derivative of the pressures, only needed for saving the data to a csv
    """
    sigma = 0.001
    dataset_size = p0.shape[0]

    p0_dot = np.zeros((dataset_size))
    p1_dot = np.zeros((dataset_size))
    p2_dot = np.zeros((dataset_size))
    p3_dot = np.zeros((dataset_size))

    for i in range(1, dataset_size):
        Ts  = (pressure_t[i] - pressure_t[i-1])
        denom = (2*sigma + Ts)
        beta = ((2*sigma - Ts) / denom)
        p0_dot[i] = (beta * p0_dot[i-1]) + (2 / denom) * (p0[i] - p0[i-1])
        p1_dot[i] = (beta * p1_dot[i-1]) + (2 / denom) * (p1[i] - p1[i-1])
        p2_dot[i] = (beta * p2_dot[i-1]) + (2 / denom) * (p2[i] - p2[i-1])
        p3_dot[i] = (beta * p3_dot[i-1]) + (2 / denom) * (p3[i] - p3[i-1])

    return p0_dot, p1_dot, p2_dot, p3_dot

def dirty_derivative_bend_angles(t, udot, vdot):
    """
    This function calculates the dirty derivative of the bend angles, only needed for saving the data to a csv
    """
    sigma = 0.001
    dataset_size = udot.shape[0]

    uddot = np.zeros((dataset_size))
    vddot = np.zeros((dataset_size))

    for i in range(1, dataset_size):
        Ts  = (t[i] - t[i-1])
        denom = (2*sigma + Ts)
        beta = ((2*sigma - Ts) / denom)
        uddot[i] = (beta * uddot[i-1]) + (2 / denom) * (udot[i] - udot[i-1])
        vddot[i] = (beta * vddot[i-1]) + (2 / denom) * (vdot[i] - vdot[i-1])

    return uddot, vddot

Run the sim

In [10]:
save_to_csv = False # set to true if you want to save sim data to csv
save_path = ''

#---------------------------------------- run sim-----------------------------------------------
#set params 
optimal_scaled_params = scaled_params #TODO: update this with your choice of scaled params
grub_model.set_params(optimal_scaled_params)

num_points = 1000 #specify number of data points to simulate #18000-1 #1000 
dt = time[1] - time[0]
sim_time = time[num_points] - time[0]
horizon = int(sim_time / dt)

# set initial x
x_sim = np.zeros((horizon, grub_model.numStates)) 
x = x_truth[0,:].reshape((1, grub_model.numStates))

fig = plt.figure()
ax = fig.add_subplot(projection="3d")
plt.ion()
grub_model.visualize(x, ax)

for i in range(0, horizon):
    x = grub_model.forward_simulate_dt(x, u[i,:], dt)
    x_sim[i, :] = x.flatten()

    if i % 5 == 0:
        grub_model.visualize(x, ax)

AE = np.abs(x_sim - x_truth[0:num_points,:])  
IAE = AE.sum(axis=0)
print(f'IAE: \n {IAE}')

x_labels = ['p0', 'p1', 'p2', 'p3', 'u_dot', 'v_dot', 'u', 'v']
u_labels = ['pc0', 'pc1', 'pc2', 'pc3']
grub_model.plot_history(x_sim, u[0:horizon,:], x_truth[0:horizon,:], x_labels, u_labels, block=False)

if save_to_csv:
    print('saving sim data to csv')
    save_to_csv(x_sim, u, horizon, sim_time, save_path)

IAE: 
 [5444.22259475 7117.64864258 5245.99405801 5473.2889056   211.65140492
  175.5822833    98.76007359  165.06680635]
